In [300]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score

%run ../src/Munger.py
%run ../src/Models.py
%run ../src/utils.py

In [6]:
# load dataset
numer_ai = pd.read_csv('../data/numerai_datasets/numerai_training_data.csv')
test = pd.read_csv('../data/numerai_datasets/numerai_tournament_data.csv')

In [7]:
# split into training and validation sets as per the competition instructions
train = numer_ai[numer_ai.validation == 0]
validation = numer_ai[numer_ai.validation == 1]

In [301]:
munger = Munger(train, validation, test)

In [302]:
## remove correlated features
munger.remove_correlated_features()

In [271]:
## label encoding a categorical feature
munger.label_encoding()

<string>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
<string>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [303]:
## one hot encoding categorical feature
munger.one_hot_encoding()

In [304]:
X_train = munger.X
y_train = munger.y

X_validation = munger.X_validation
y_validation = munger.y_validation

X_test = munger.X_test

In [306]:
X_train.head()

,f1,f2,f3,f4,f6,f10,f7,c1_1,c1_10,c1_11,...,c1_22,c1_23,c1_24,c1_3,c1_4,c1_5,c1_6,c1_7,c1_8,c1_9
0,134182390,97300587,135824563,90767607,65148427,92106582,109834029,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,111610401,132186734,83887688,120772423,100560251,134884316,69916843,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,72896251,113365043,72912608,126007508,74975809,116126096,87833921,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,113253706,96665624,89391907,77862818,100094204,86510339,116150057,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,122837395,71633727,125829672,73401946,71572170,68026428,127205376,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [275]:
model = Models()
clf = model.logistic_regression_model()

In [276]:
# fit a model
clf.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0))])

In [277]:
## split into validation and testing set

from sklearn.cross_validation import train_test_split

X_val, X_t, y_val, y_t = train_test_split(X_validation, y_validation, 
                                                test_size=0.33, random_state=44)

In [278]:
## predictions
predsTrain = clf.predict_proba(X_train)[:, 1]
predsValidation = clf.predict_proba(X_val)[:, 1]
predsTest = clf.predict_proba(X_t)[:, 1]

In [279]:
## check to see how this dummy classifier performs
print 'ROC AUC Score on the training examples %f ' %(roc_auc_score(y_train, predsTrain))
print 'ROC AUC Score on the validation examples %f ' %(roc_auc_score(y_val, predsValidation))
print 'ROC AUC Score on the validation examples %f ' %(roc_auc_score(y_t, predsTest))

ROC AUC Score on the training examples 0.528199 
ROC AUC Score on the validation examples 0.525975 
ROC AUC Score on the validation examples 0.539896 


In [280]:
# create full dataset
munger.concatenate_train_validation()

In [281]:
# full dataset
X_full = munger.X_full
y_full = munger.y_full

In [284]:
## fit on whole dataset
clf.fit(X_full, y_full)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0))])

In [285]:
predictions = clf.predict_proba(X_test)[:, 1]

In [286]:
# submission dataframe
submission_df = pd.read_csv('../data/numerai_datasets/numerai_example_predictions.csv')

In [287]:
prepare_submission(submission_df, predictions, 'logistic_with_categorical.csv')